In [1]:
import numpy as np 
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re

In [2]:
def tagAndTrainingData(file):
	#returning tokenize sentence
	tags = []
	documents = []
	counter = 1
	with open(file) as f:
		for line in f:
			#skip the first line
			if counter == 1:
				counter += 1
				continue

			tags.append(line[:3]) #separating document_id
			documents.append(line[3:]) #separating body of the document
	return tags,documents

In [9]:
Y,X = tagAndTrainingData('../preprocessing/trainingdataset.txt')
df_X = pd.Series(X,index=None)
df_Y = pd.Series(Y,index=None)

In [10]:
df_X.head()

0    CARGILL UK STRIKE TALKS BREAK OFF WITHOUT RESU...
1    UK GRAIN/POTATO FUTURES VOLUME DOWN IN FEBRUAR...
2    EUROPEAN SOY/FEED MARKET OPENS QUIETLY ROTTERD...
3    DUTCH SOYMEAL IMPORTS FALL IN JANUARY HEERLEN,...
4    FALLING SOYBEAN CRUSH RATIOS CUT OUTPUT AUTHOR...
dtype: object

In [11]:
#for extracting keyword we going to use RAKE algorithm
#run rake algorithm for each document and extract the important terms from the document
from rake_nltk import Rake
from nltk.tokenize import word_tokenize

In [16]:
def preprocess_the_document(stopwords,document):
    word_tokens = word_tokenize(document)
    filtered_sentence = [word for word in word_tokens if not word in stopwords] #return as list
    #join the document without the stopwords
    filtered_document = ' '.join(filtered_sentence)
    return filtered_document

The output from Rake Algorithm is as a List.so we have to join the extracted keyword for each Document

In [44]:
#before extracting keywords from the document we want to preprocess the dataset
r = Rake() #create object for Rake
keywords_list = []
stopset = nltk.corpus.stopwords.words('english')
i=1
for documents in df_X:
    preprocessed_docs = preprocess_the_document(stopset,documents)#preprocessed document
    r.extract_keywords_from_text(preprocessed_docs)
    ranked_phrase = r.get_ranked_phrases() #type as a list
    #join the list
    join_ranked_phrase_docs = " ".join(ranked_phrase)
    keywords_list.append(join_ranked_phrase_docs)
    if i%1000 == 0:
        print("Finished Extracting Keywords from {} documents".format(i))
    i+=1 #increment the document

Finished Extracting Keywords from 1000 documents
Finished Extracting Keywords from 2000 documents
Finished Extracting Keywords from 3000 documents
Finished Extracting Keywords from 4000 documents
Finished Extracting Keywords from 5000 documents
Finished Extracting Keywords from 6000 documents
Finished Extracting Keywords from 7000 documents
Finished Extracting Keywords from 8000 documents
Finished Extracting Keywords from 9000 documents
Finished Extracting Keywords from 10000 documents
Finished Extracting Keywords from 11000 documents


In [45]:
len(keywords_list)

11475

In [46]:
keywords_list[1]

"march traded volumes uk grain potato futures february previous month pigmeat pig cash settlement futures higher value declined nine mln stg mln nine pigmeat contracts traded february tonnes registered main crop potato futures february valued mln stg stg pig cash settlement futures saw contracts traded official figures show combined wheat barley futures trade declined mln stg january soymeal futures trade totalled value fell mln stg mln potato futures volume six previous month grain feed trade association uk grain tonnes mln january february london figures show stg '' value rose stg valued tonnes representing carcases january versus total gafta"

In [47]:
document_id = []
for docs in df_Y:
    document_id.append(docs)

In [48]:
#save in a file for future reference
len(document_id)

11475

In [49]:
#create as a dataFrame
df = pd.DataFrame({'Document_id':document_id,'Keywords':keywords_list})

In [50]:
#save as txt file
df.to_csv('features_2.txt',sep=",",index=None)